In [5]:
from customtkinter import *
from PIL import Image, ImageTk
from tests import PrimeTest

In [6]:
class SmallFrame(CTkFrame):
    def __init__(self, master, main_master, **kwargs):
        super().__init__(master, **kwargs)
        self.algo_label = CTkLabel(self, text="Algorithm:", fg_color="transparent")
        self.algo_label.grid(row=0, column=0, padx=5, pady=0)
        
        self.combobox = CTkComboBox(self, values=["Automatically", "Trial Divisions", "Solovay-Strassen", "Miller", "Miller-Rabin", "Adleman-Pomerans-Rumeli"], command=lambda choice, arg=main_master: self.combobox_callback(choice, arg), state="readonly")
        self.combobox.set("Automatically")
        self.combobox.grid(row=0, column=1, padx=5, pady=0)
        main_master.main_frame.message_button = CTkButton(main_master.main_frame, text="", command=self.message_button_click, width=26, height=26, state="disabled", fg_color="transparent", text_color_disabled=("black", "white"))
        main_master.main_frame.message_button.grid(row=3, column=0, columnspan=2, padx=0, pady=0)
            
        SmallFrame.message_for_automatically(main_master)
        
    def combobox_callback(self, choice, master):
        if(self.combobox.get() != "Automatically"):
            master.main_frame.message_button.configure(text="")
            warning_ico = CTkImage(Image.open("assets/warning.png"), size=(32, 32))
            master.main_frame.message_button.configure(text="Warning: if you don't know what you are doing, it's recommended to select back the \n\"Automatic\" option, because independent and inconsiderable choice of algorithm could \nlead to unforeseen situations, such as lagging, freezing and crashing of the application.", image=warning_ico)
        else:
            SmallFrame.message_for_automatically(master)
          
    @staticmethod
    def message_for_automatically(master):
            master.main_frame.message_button.configure(text="")
            accept_ico = CTkImage(Image.open("assets/accept.png"), size=(24, 24))
            master.main_frame.message_button.configure(text="Have fun with prime checker and test if your number is prime with \"Automatic\" algorithm\n calculation for your number.\n", image=accept_ico)
            
    def message_button_click(self):
        ... # never used
    
    def get_combobox_value(self):
        return self.get()
        
class PalettesFrame(CTkFrame):
    def __init__(self, master, **kwargs):
        super().__init__(master, **kwargs)
        
        self.bright_theme_button = CTkButton(self, text="", command=self.bright_theme_button_click, width=26, height=26, fg_color="white")
        self.bright_theme_button.grid(row=0, column=0, padx=5, pady=0)

        self.dark_theme_button = CTkButton(self, text="", command=self.dark_theme_button_click, width=26, height=26, fg_color="black")
        self.dark_theme_button.grid(row=0, column=1, padx=5, pady=0)
    
    def bright_theme_button_click(self):
        set_appearance_mode("light")
        
    def dark_theme_button_click(self):
        set_appearance_mode("dark")

class HeaderFrame(CTkFrame):
    def __init__(self, master, **kwargs):
        super().__init__(master, **kwargs)
        
        self.small_frame = SmallFrame(master=self, main_master=master)
        self.small_frame.grid(row=0, column=0, padx=20, pady=10)
        
        self.palletes_frame = PalettesFrame(master=self)
        self.palletes_frame.grid(row=0, column=1, padx=68, pady=10)
        
        info_ico = CTkImage(Image.open("assets/info.png"), size=(16, 16))
        self.info_button = CTkButton(self, text="", command=self.info_button_click, width=26, height=26, image=info_ico)
        self.info_button.grid(row=0, column=2, padx=40, pady=10)
        
    def info_button_click(self):
        print("Info button click")
        
# class ScrollableFrame(CTkScrollableFrame):
#     def __init__(self, master, **kwargs):
#         super().__init__(master, **kwargs)

#         self.label = CTkLabel(self)
#         self.label.grid(row=0, column=0, padx=0)

class MainFrame(CTkFrame):
    def __init__(self, master, **kwargs):
        super().__init__(master, **kwargs)
        self.grid_rowconfigure(0, weight=1)
        self.grid_columnconfigure(0, weight=1)
        self.grid_columnconfigure(1, weight=1)
        
        self.entry_for_num_to_check = CTkEntry(self, placeholder_text="Enter the number...", width=426)
        self.entry_for_num_to_check.grid(row=0, column=0, padx=20, pady=10)
        
        self.check_button = CTkButton(self, text="Check", width=72, text_color="white")
        self.check_button.grid(row=0, column=1, padx=20, pady=10)
        
        self.check_button.bind("<Button-1>", lambda event, arg=master: self.check_button_click(event, arg))
        master.bind("<Return>", lambda event, arg=master: self.check_button_click(event, arg))
        
        self.answer_label = CTkLabel(self, text="", fg_color="transparent")
        self.answer_label.grid(row=2, column=0, columnspan=2, padx=0, pady=0)
        
        # self.scrollable_frame = ScrollableFrame(master=self, width=514, height=430)
        # self.scrollable_frame.grid(row=3, column=0, columnspan=2, padx=10, pady=5)
    
    def check_button_click(self, event, master):
        self.answer_label.configure(text="")
        try:
            str_num = self.entry_for_num_to_check.get()
            if len(str_num) == 0:
                ...
            elif str_num.isspace():
                self.entry_for_num_to_check.delete(0, len(str_num))
            else:
                while (str_num[0] == '0'):
                    str_num = str_num[1:]
                num = int(str_num)
                if num <= 0:
                    self.answer_label.configure(text="Prime numbers are positive, non-zero numbers that have exactly two factors.", text_color="red")
                    # self.entry_for_num_to_check.delete(0, len(str_num))
                elif num == 1:
                    self.answer_label.configure(text="Not a prime number, neither composite.", text_color=("black", "white"))
                    # self.entry_for_num_to_check.delete(0, len(str_num))
                else:
                    current_algorithm = master.header_frame.small_frame.combobox.get()
                    match current_algorithm:
                        case "Trial Divisions":
                            res = PrimeTest.trial_divisions(num)
                            if res:
                                self.answer_label.configure(text="Your number is prime.", text_color="green")
                            else:
                                self.answer_label.configure(text="Your number is composite.", text_color=("black", "white"))
                        case "Solovay-Strassen":
                            res = PrimeTest.solovay_strassen(num, 10)
                            if res:
                                self.answer_label.configure(text="Your number is prime.", text_color="green")
                            else:
                                self.answer_label.configure(text="Your number is composite.", text_color=("black", "white"))
                        case "Miller":
                            res = PrimeTest.miller(num)
                            if res:
                                self.answer_label.configure(text="Your number is prime.", text_color="green")
                            else:
                                self.answer_label.configure(text="Your number is composite.", text_color=("black", "white"))
                        case "Miller-Rabin":
                            res = PrimeTest.miller_rabin(num)
                            if res:
                                self.answer_label.configure(text="Your number is prime.", text_color="green")
                            else:
                                self.answer_label.configure(text="Your number is composite.", text_color=("black", "white"))
                        case "Adleman-Pomerans-Rumeli":
                            res = PrimeTest.APRtest(num)
                            if res:
                                self.answer_label.configure(text="Your number is prime.", text_color="green")
                            else:
                                self.answer_label.configure(text="Your number is composite.", text_color=("black", "white"))
                        case _:
                            ... # never happened
        except:
            self.answer_label.configure(text="Invalid input. You must avoid non-numeric characters and spaces.", fg_color="transparent", text_color="red")
            self.entry_for_num_to_check.delete(0, len(str_num))

class App(CTk):
    def __init__(self):
        super().__init__()
        set_appearance_mode("dark")
        self.geometry("600x600")
        self.title("Prime checker")

        self.main_frame = MainFrame(master=self)
        self.main_frame.grid(row=1, column=0, padx=10, pady=0)
        
        self.header_frame = HeaderFrame(master=self, width=580, height=30)
        self.header_frame.grid(row=0, column=0, padx=10, pady=5)

app = App()
app.iconbitmap('assets\\favicon.ico')
app.resizable(False, False)
app.mainloop()

1
1
1
